In [144]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [145]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse

In [382]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *

In [180]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df),len(test_users_with_city))

16967 1252801 16967


In [184]:
reviews, user_agg, org_agg = load_reviews(users_df=users_df, orgs_df=orgs_df)
orgs_df = orgs_df.merge(org_agg, on='org_id')
users_df = users_df.merge(user_agg, on='user_id', how='left')

In [181]:
test_users_df = test_users_with_city\
    .merge(reviews[reviews.rating>=4.0][['user_id','org_id']], on='user_id', how='left')\
    .groupby(['user_id','city'])\
    .aggregate(list)\
    .reset_index()

In [399]:
train_reviews, test_reviews = train_test_split(reviews, users_df, 
                                               min_user_reviews=2,
                                               min_ts=700,
                                               frac=0.5)
print(len(train_reviews), len(test_reviews))

3617059 14642


In [400]:
result, encs = prepare_reviews_i2i(train_reviews, users_df, orgs_df,
                                   min_reviews_per_user=2,
                                   min_org_reviews=30,
                                   min_travels_reviews=2,
                                   min_org_score=4.0)

In [401]:
m = reviews_matrix(result, encs)
cc_mat = m.T * m

In [413]:
ease_sim = ease_solution(cc_mat, l2=10000).T
ease_preds = i2i_predict(ease_sim, test_reviews, encs, N=20)
print(recall(test_reviews, ease_preds))

0.1464527407630298


In [410]:
for N in [10,20,30,40,50,60]:
    print(f"{N} {recall(test_reviews, ease_preds, N=N)}")

10 0.12779335415210258
20 0.16014120758836226
30 0.1844125330840387
40 0.20291219682932518
50 0.21752142996851456
60 0.23092607506618235


In [408]:
for t_size in [1,2,4,5,10,20,30]:
    test_preds = test_reviews[test_reviews.org_id.str.len()>=t_size]
    print(f"{t_size} {recall(test_preds, ease_preds)}")

1 0.16014120758836226
2 0.17813930834511535
4 0.19089142542699192
5 0.1928852276267624
10 0.18506508303086844
20 0.18265661224463817
30 0.18530712410406147


In [409]:
for t_size in [1,2,4,5,10,20,30]:
    test_preds = test_reviews[test_reviews.org_id.str.len()<=t_size]
    print(f"{t_size} {recall(test_preds, ease_preds)}")

1 0.09611995220431883
2 0.11318625081101402
4 0.13359287771305287
5 0.14108864717746197
10 0.15430611815757125
20 0.15830731465570466
30 0.15940066005806816


In [337]:
recall(ease_preds, J_preds),recall(J_preds,ease_preds)

(0.4429693997227922, 0.4440867657696127)

In [237]:
validate_preds(fallback_with_top_recs(ease_preds, orgs_df), orgs_df, users_df)

All good


In [241]:
save_predictions(fallback_with_top_recs(merged, orgs_df), path='merged_ans.csv')

In [219]:
save_predictions(fallback_with_top_recs(pmi_preds, orgs_df), path='pmi_ans.csv')

In [248]:
save_predictions(fallback_with_top_recs(ease_preds, orgs_df), path='ease_ans.csv')

In [196]:
save_predictions(fallback_with_top_recs(J_preds,orgs_df), path='J_ans.csv')

In [23]:
aspects = load_aspects()
rubrics = load_rubrics(orgs_df=orgs_df)
features = load_features(orgs_df=orgs_df)